In [31]:
from temporal_grounding.temporal_grounding import extract_timestamps_from_video

vp = r"testing\shot_retrevial\videos\940432_2900_KK MZT Skopje Aerodrom_2953_KK Buducnost_period2.avi"
sp = r"testing\shot_retrevial\timestamps\940432_2900_KK MZT Skopje Aerodrom_2953_KK Buducnost_period2.json"
extract_timestamps_from_video(vp, sp)

Finding time-remaining ROI for video at testing\shot_retrevial\videos\940432_2900_KK MZT Skopje Aerodrom_2953_KK Buducnost_period2.avi


 90%|█████████ | 540/600 [00:30<00:03, 17.57it/s]


Extracting timestamps for video at testing\shot_retrevial\videos\940432_2900_KK MZT Skopje Aerodrom_2953_KK Buducnost_period2.avi


100%|██████████| 600/600 [00:21<00:00, 27.63it/s]


In [12]:
from temporal_grounding.temporal_grounding import convert_float_to_time_str, convert_time_to_float
from data_prep import *
from datascience import *
import numpy as np

log_path = get_log_path("972173")
log_path

start_time = 600
# convert_float_to_time_str(531.199)
convert_time_to_float("5:13")

# arr = []
# with open(log_path) as file:
#     doc = csv.reader(file, delimiter=';')
#     for row in doc:
#         arr.append(row)

# for row in arr[1: ]:
#     if row[2][0] == "2" or row[2][0] == "3":
#         print("Shot", row[26], "at", row[23])


313.0

In [2]:
import cv2

def clip_video(input_path, output_path, start_frame, end_frame):
    try:
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            raise Exception("Error: Could not open video file")
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)
            if cap.get(cv2.CAP_PROP_POS_FRAMES) == end_frame:
                break
        cap.release()
        out.release()
    except Exception as e:
        print(f"Error: {str(e)}")

In [11]:
from data_prep import *
import os
import json

video_path = r"testing\shot_retrevial\videos\940432_2900_KK MZT Skopje Aerodrom_2953_KK Buducnost_period2.avi"
shots_out_path = r"testing\shot_retrevial\shot_clips"
timestamps_path = r"testing\shot_retrevial\timestamps\940432_2900_KK MZT Skopje Aerodrom_2953_KK Buducnost_period2.json"

def extract_shots_from_video(video_path, shots_out_dir, timestamps_path=None):

    try:
        game_id = os.path.basename(video_path).split("_")[0]
    except:
        raise Exception(f"Bad video path: {video_path}")
    try:
        with open(timestamps_path, "r") as f:
            timestamps = json.load(f)
    except:
        raise Exception(f"Error: bad timestamps path: {timestamps_path}.")

    path_to_logs = get_log_path(game_id)
    shot_events = get_shot_events(path_to_logs)
    intervals = []
    for event in shot_events:
        key = f"{event[2]}_{event[3]}"
        if key in timestamps:
            try:
                backwards_pad = 15
                shot_duration = 30 * 3
                start_frame = timestamps[key] - backwards_pad
                end_frame = start_frame + shot_duration
                intervals.append([start_frame, end_frame, event])
            except:
                pass

    shot_index = 0
    video_title = os.path.basename(video_path)
    shot_subdir = os.path.join(shots_out_dir, video_title.split(".")[0])
    os.makedirs(shot_subdir, exist_ok=True)
    for interval in intervals:
        start_frame, end_frame = interval[0], interval[1]
        is_shot_made = interval[2][1] 
        output_path = os.path.join(shot_subdir, f"_clip_{shot_index}_{is_shot_made}_.avi")
        print(f"Shot clips saved to: {output_path}")
        clip_video(video_path, output_path, start_frame, end_frame)
        shot_index += 1
        

extract_shots_from_video(video_path, shots_out_path, timestamps_path)

Shot clips saved to: testing\shot_retrevial\shot_clips\940432_2900_KK MZT Skopje Aerodrom_2953_KK Buducnost_period2\_clip_0_True_.avi
Shot clips saved to: testing\shot_retrevial\shot_clips\940432_2900_KK MZT Skopje Aerodrom_2953_KK Buducnost_period2\_clip_1_True_.avi


In [2]:
( 6 * 60 ) + 30

390